import

In [1]:
# === Cell 1: Imports ===

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
import hopsworks
import config


In [ ]:
import os
os.environ["HF_API_KEY"] = ""

In [3]:
# === Cell 2: Connect to Hopsworks ===

from config import HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        # project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )
fs = project.get_feature_store()


2026-01-08 02:06:49,443 INFO: Initializing external client
2026-01-08 02:06:49,443 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-08 02:06:51,077 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286333


In [4]:
# === Cell 3: Load Feature Views ===

metadata_fv = fs.get_feature_view(
    name="paper_metadata_fv",
    version=1,
)

chunk_fv = fs.get_feature_view(
    name="paper_chunk_fv",
    version=1,
)

metadata_fv.init_serving(1)
chunk_fv.init_serving(1)


In [5]:
# === Cell 4: Load Embedding Model ===
sentence_transformer = SentenceTransformer(
    config.EMBEDDING_MODEL_NAME
)

2026-01-08 02:07:05,063 INFO: Use pytorch device_name: cpu
2026-01-08 02:07:05,064 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [7]:
# === Cell 5: Similarity Search Engine ===

from functions.similarity_search import SimilaritySearchEngine

search_engine = SimilaritySearchEngine(
    embedding_model=sentence_transformer,
    metadata_feature_view=metadata_fv,
    chunk_feature_view=chunk_fv,
)


In [8]:
# === Cell 6: Context Builder ===

from functions.context_builder import ContextBuilder

context_builder = ContextBuilder(
    max_tokens=2000,
    max_chunks=8,
)


In [9]:
# === Cell 7: Prompt Synthesizer ===

from functions.prompt_synthesis import PromptSynthesizer

prompt_synthesizer = PromptSynthesizer()


In [10]:
# === Cell 8: MCP Dispatcher ===

from functions.mcp_dispatcher import MCPDispatcher

mcp_dispatcher = MCPDispatcher(
    search_engine=search_engine
)


In [ ]:
# === Cell 9: Agentic Inference ===

from functions.agent_loop import AgenticInference
from functions.llm_wrapper import LLMWrapper  

llm = LLMWrapper(
    model_name_or_path="mistralai/Mistral-7B-v0.1"  
)

agent = AgenticInference(
    llm=llm,
    search_engine=search_engine,
    context_builder=context_builder,
    prompt_synthesizer=prompt_synthesizer,
    mcp_dispatcher=mcp_dispatcher,
)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

2026-01-08 02:07:49,503 WARNING: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute

2026-01-08 02:07:49,505 WARNING: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


2026-01-08 02:07:49,804 WARNING: Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

2026-01-08 02:07:52,900 WARNING: Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


2026-01-08 02:07:52,905 WARNING: Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

In [ ]:
# === Cell 10: Run Agent ===

query = "What are the best risk reporting practices?"

answer = agent.run(query)
print(answer)
